Please start by making your own copy of this Google Colab notebook. You can upload your finished version [here](https://drive.google.com/drive/folders/16jdlUw6f_oOndT9IE7FlTM4Y60Vxu7cK?usp=sharing). 

Learning objectives:  

* Implement a model-free agent
* Implement a model-based agent
* Compare their behavior under different transition dynamics

The cell below imports some necessary modules:

In [ ]:
import numpy as np  
from scipy.stats import norm

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import pandas as pd

# Define environment

In [ ]:
class SimpleMaze(object):
  
  """Class for the simple maze environment from lecture, with a slip probability. 
  This is a 3-state maze in which the agent makes a two-step decision whether 
  to go left or right. 
  
  Parameters
  ----------
  p_slip : float, (0, 1)
      The probability that the agent "slips": that is, it goes right if it selects 
      the left action, or left if it selects the right action
      
      Note: this applies only to the starting state. 
 
  """

  def __init__(self, p_slip):

    # initialize state space
    # state 1 = START DOOR, state 2 = LEFT DOOR, state 3 = RIGHT DOOR
    self.n_states = 3
    self.state_space = list(np.arange(self.n_states)+1) + [0] # 0 is terminal state

    # initialize action space
    self.action_space = [1, 2]
    self.k = len(self.action_space)
   
    # initialize parameters 
    self.p_slip = p_slip    # transition probability

    # initialize timestep and state
    self.t = 0
    self.state = 1  

    print('timestep: ' + str(self.t))

  def reset(self):

    # re-initialize timestep
    self.t = 0

    # resets environment to initial state
    return self.state_space[0]
  
  def step(self, state, action, verbose=True):

    assert state in self.state_space, "Invalid state" 
    assert action in self.action_space, "Invalid action" 


    if verbose == True:

      if state == 1:
        print('at start door')

      if action == 1:
        print('going left...')
      else:
        print('going right...')

    # the step method takes as input a state and action and changes the environment
    # action 1 = LEFT, action 2 = RIGHT 

    if state == 1:         # at start door 

        # generate random number between 0 and 1.
        # this draws from a uniform distribution, so we have an equal probability
        # of generating any real number between 0 and 1.
        p = np.random.rand()
        
        if action == 1:    # agent picks left 

          # check if agent slipped
          if p < self.p_slip: 

            print('oops, slipped!')

            new_state = 3   
          else: 
            new_state = 2
        
        else:              # agent picks right
          
          # check if agent slipped
          if p < self.p_slip: 

            print('oops, slipped!')

            new_state = 2   
          else: 
            new_state = 3
        
        reward = 0

    elif state == 2:      # reached left door
        
        if action == 1:   # agent picks left
         
          new_state = 0
          reward = 4
        
        else:             # agent picks right
          
          new_state = 0
          reward = 0
    
    else:                 # reached right door

        if action == 1:   # agent picks left
          
          new_state = 0
          reward = 2
        
        else:             # agent picks right
          
          new_state = 0
          reward = 1

    # increment timestep
    self.t = self.t + 1

    if verbose == True:

      print('reward: ' + str(reward))
      print('new state: ' + str(new_state))
      
      if new_state == 2: 
        print('reached left door')
      if new_state == 3: 
        print('reached right door')
      if new_state == 0: 
        print('reached terminal state')

      print('timestep: ' + str(self.t))

    return reward, new_state


# Test environment

In [ ]:
# Determinisic case
p_slip = 0

maze = SimpleMaze(p_slip)

print("state space:")
print(maze.state_space)

print("action space:")
print(maze.action_space)

state = maze.reset()

# Going left twice in a row
r, new_state = maze.step(state, 1)
r, new_state = maze.step(new_state, 1)

timestep: 0
state space:
[1, 2, 3, 0]
action space:
[1, 2]
at start door
going left...
reward: 0
new state: 2
reached left door
timestep: 1
going left...
reward: 4
new state: 0
reached terminal state
timestep: 2


In [ ]:
# Probabilistic case
p_slip = 0.9

maze = SimpleMaze(p_slip)

print("state space:")
print(maze.state_space)

print("action space:")
print(maze.action_space)

state = maze.reset()

# Go left twice in a row
r, new_state = maze.step(state, 1)
r, new_state = maze.step(new_state, 1)

timestep: 0
state space:
[1, 2, 3, 0]
action space:
[1, 2]
at start door
going left...
oops, slipped!
reward: 0
new state: 3
reached right door
timestep: 1
going left...
reward: 2
new state: 0
reached terminal state
timestep: 2


# Exercise 1: model-free learning 
Let's start by defining a Q-Learning agent that learns to navigate the maze from experience with no knowledge of the model.  

In [ ]:
class QLearning(object):
  """Class for the Q-learning algorithm. 

  Parameters
  ----------

  alpha : float, range (0, 1)
      Learning rate.

  gamma : float, range (0, 1)
      Discount factor.

  epsilon : float, range (0, 1)
      Epsilon probability of exploration.

  """

  def __init__(self, env, alpha, gamma, epsilon, model_based=False, q_init=False):

    # initialize action space
    self.action_space = env.action_space

    self.a = alpha
    self.g = gamma
    self.eps = epsilon

    self.model_based = model_based

     # initialize Q-values 
    if q_init: # check if initial q-values were provided 
      self.q = np.ones((env.n_states+1, env.k))*q_init     
    else:   
      self.q = np.zeros((env.n_states+1, env.k))

  def policy(self,state):

    # in the epsilon-greedy case, the agent has an internal representation
    # of the value of each action. its policy is to pick the action with the 
    # highest value with probability 1-epsilon and explore a random action with 
    # probabilty epsilon.  

    # generate random number between 0 and 1.
    # this draws from a uniform distribution, so we have an equal probability
    # of generating any real number between 0 and 1.
    p = np.random.rand()

    # select action
    if (p < self.eps): # is the number we drew is smaller than epsilon? 
      action = np.random.choice(env.k)+1 # random action
    else: 
      # choose argmax, breaking ties randomly 
      action = np.random.choice(np.flatnonzero(self.q[state-1] == self.q[state-1].max())) + 1
    
    return action

  def update(self, current_state, action, reward, new_state, verbose=False):

    # we are at the start door 
    if current_state == 1:

      if self.model_based: 

        # model-based learning
        # planning to go left
        self.q[current_state-1, 0] = (1-env.p_slip) * max(self.q[1,:]) + env.p_slip * max(self.q[2,:])
        # planning to go right
        self.q[current_state-1, 1] = env.p_slip * max(self.q[1,:]) + (1-env.p_slip) * max(self.q[2,:])
      
        
      else: 
        # model-free learning
        self.q[current_state-1, action-1] = self.q[current_state-1, action-1] + self.a*(reward + self.g * max(self.q[new_state-1, :]) - self.q[current_state-1, action-1])
  
    # we are at one of the second doors
    elif (current_state == 2) | (current_state == 3):

      # model-free learning
      self.q[current_state-1, action-1] = self.q[current_state-1, action-1] + self.a*(reward + self.g * max(self.q[new_state-1, :]) - self.q[current_state-1, action-1])

    if verbose == True: 
      print(self.q)
  

## **1A** 

Write a simulation that runs the Q-Learning agent for 1000 episodes in a deterministic maze environment ($p_{slip} = 0$), with a learning rate of $0.2$, no discounting, and exploration probability of $0.2$. Plot the Q-Values as a function of episode. What policy does the agent learn at convergence (i.e. what actions will it take in every state?)? 

In [ ]:
def run_simulation(env_params, agent_params, n_episodes):

  """Function for running a simulation of Q learning agent in the simple maze environment."""

  # make environment
  env = SimpleMaze(env_params['p_slip'])
  current_state = env.reset() #move reset to beginning not within loop


  # initialize agent 
    agent = QLearning(env, 
                  agent_params['learning_rate'], 
                  agent_params['discount_factor'], 
                  agent_params['explore_prob'], 
                  agent_params['model_based'])

 # initialize output containers    
  Q_1_left = []  # state 1, left
  Q_1_right = [] # state 1, right

  Q_2_left = []  # state 2, left
  Q_2_right = [] # state 2, right

  Q_3_left = []  # state 3, left
  Q_3_right = [] # state 3, right
 


  for e in np.arange(n_episodes):

    #got rid of inner loop- just hard coding 3 levels

    
    #first level 
    action = agent.policy(current_state) # agent takes an action
    reward, new_state = env.step(current_state, action, verbose=False)        # environment returns reward and new state
    agent.update(current_state, action, reward, new_state, verbose=False)     # agent updates q-values
    current_state = new_state # current state becomes new state


    #second level
    action = agent.policy(current_state) # agent takes an action
    reward, new_state = env.step(current_state, action, verbose=False)        # environment returns reward and new state
    agent.update(current_state, action, reward, new_state, verbose=False)     # agent updates q-values
    
    # Store q-values for plotting
    Q_1_left.append(agent.q[0,0])  # state 1, left
    Q_1_right.append(agent.q[0,1]) # state 1, right

    Q_2_left.append(agent.q[1,0])  # state 2, left
    Q_2_right.append(agent.q[1,1]) # state 2, right

    Q_3_left.append(agent.q[2,0])  # state 3, left
    Q_3_right.append(agent.q[2,1]) # state 3, right

    current_state = env.reset() # current state becomes new state

  return Q_1_left, Q_1_right, Q_2_left, Q_2_right, Q_3_left, Q_3_right
  

In [ ]:
env_params = {
      'p_slip': 0
  }

agent_params = {
      'learning_rate': 0.2,
      'discount_factor': 0,
      'explore_prob': 0.2,
      'model_based': False
  }

n_episodes = 100

Q_1_left, Q_1_right, Q_2_left, Q_2_right, Q_3_left, Q_3_right = run_simulation(env_params, agent_params, n_episodes)


# Plot as a function of episode
fig, ax = plt.subplots(1,1,figsize=(8,6))
plt.plot(np.array(Q_1_left), label='state 1, left');
plt.plot(np.array(Q_1_right), label='state 1, right');
plt.plot(np.array(Q_2_left), label='state 2, left');
plt.plot(np.array(Q_2_right), label='state 2, right');
plt.plot(np.array(Q_3_left), label='state 3, left');
plt.plot(np.array(Q_3_right), label='state 3, right');
plt.xlabel('Episode');
plt.ylabel('Value');
plt.title('Model free, deterministic');
ax.legend();

timestep: 0
Ran the maze 0 times so far
state space:
[1, 2, 3, 0]
2
action space:
1
going left...
reward: 4
new state: 0
reached terminal state
timestep: 1
reward: 4
new state: 0
[[0. ]
 [0.8]
 [0. ]
 [0. ]]
3
action space:
3


AssertionError: ignored

In [ ]:
# Value
df_wide = pd.DataFrame(V.T)
df_wide.loc[len(df_wide.index)-1] = 10

fig, ax = plt.subplots(1,1,figsize=(9,4))
sns.lineplot(data=df_wide, legend=True, palette='Blues', markers=True, dashes=False)
ax.legend(labels=list(np.arange(params['n_states'])+1))
ax.axvline(x=params['n_states']-1, color='blue',linewidth=4)
ax.set_xticks(np.arange(params['n_states']))
labels = [str(i) for i in np.arange(params['n_states'])+1]
labels[-1] = 'R'
ax.set_xticklabels(labels)
ax.set_xlabel('State',fontsize=15);
ax.set_ylabel('Value',fontsize=15);
sns.set_style('white');
sns.despine();

# TD-error
df_wide = pd.DataFrame(D.T)

fig, ax = plt.subplots(1,1,figsize=(9,4))
sns.lineplot(data=df_wide, legend=True, palette='Greens', markers=True, dashes=False)
ax.legend(labels=list(np.arange(params['n_states'])+1))
ax.axvline(x=params['n_states']-1, color='green',linewidth=4)
ax.set_xticks(np.arange(params['n_states']))
labels = [str(i) for i in np.arange(params['n_states'])+1]
labels[-1] = 'R'
ax.set_xticklabels(labels)
ax.set_xlabel('State',fontsize=15);
ax.set_ylabel('Temporal-Difference error',fontsize=15);
sns.set_style('white');
sns.despine();

env.visualize()

##**1B** 

Repeat the above, but this time under a probabilistic environment ($p_{slip} = 0.7$). Does the agent's policy change? Why or why not?

# Exercise 2: model-based learning



## **2A**  
Let's now assume that the transition model (ie, the probabilities of going from *State 1* to *State 2* or *State 3* given the action) is known to the agent. Modify the update function of the agent to take into account the probability that the agent will slip in *State 1*. Hint: how can you use the slip probability and the learned values at the 2nd and 3rd door to plan and make choices at the start door?

Write a simulation function that runs this new agent with the same parameters as before, and plot the Q-Values at convergence. Does the agent's policy change relative to the model-free case? Why or why not? 

## **2B**  
Suppose the agent doesn't know in advance the probability that it will slip. How could it learn this from experience? Give below an update rule for the model of the environment. Modify the update function of the agent to include this update rule. Hint: think about the successor representation (SR). 